In [59]:
import numpy as np
import pandas as pd 
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pickle
from sklearn.feature_selection import SelectFromModel 
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2

In [60]:
np.random.seed(1017)
tqdm.pandas()

In [61]:
user_data = pd.read_csv('训练数据集/训练数据集_tag.csv')
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39923 entries, 0 to 39922
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               39923 non-null  object
 1   flag                             39923 non-null  int64 
 2   gdr_cd                           39923 non-null  object
 3   age                              39923 non-null  int64 
 4   mrg_situ_cd                      39923 non-null  object
 5   edu_deg_cd                       27487 non-null  object
 6   acdm_deg_cd                      39922 non-null  object
 7   deg_cd                           18960 non-null  object
 8   job_year                         39923 non-null  object
 9   ic_ind                           39923 non-null  object
 10  fr_or_sh_ind                     39923 non-null  object
 11  dnl_mbl_bnk_ind                  39923 non-null  object
 12  dnl_bind_cmb_lif_ind            

In [62]:
for col in user_data.columns:
    print(user_data[col].value_counts())

UE238AA    1
UC48A0F    1
UABF08A    1
U7E37EB    1
UE37A96    1
          ..
UCFBB34    1
U2D04C6    1
UC8EA36    1
U2C6F9E    1
U7D7DBF    1
Name: id, Length: 39923, dtype: int64
0    30970
1     8953
Name: flag, dtype: int64
M     23015
F     16172
\N      736
Name: gdr_cd, dtype: int64
30    2513
31    2462
32    2239
33    2227
29    2117
28    1998
34    1908
27    1862
26    1671
25    1542
35    1530
36    1468
38    1379
37    1323
24    1197
39    1159
41     958
40     932
23     820
42     806
22     678
43     644
44     607
45     557
46     544
47     489
49     479
48     470
50     450
21     441
52     331
51     328
20     233
57     208
53     200
54     192
56     189
55     163
58     138
60      71
59      69
62      52
61      48
63      47
65      30
64      29
66      28
19      18
67      17
69      15
68      14
70      13
71       7
74       5
72       2
73       2
76       1
77       1
84       1
79       1
Name: age, dtype: int64
A     19119
B     18598
O

In [5]:
trd_data = pd.read_csv('训练数据集/训练数据集_trd.csv')
trd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1367211 entries, 0 to 1367210
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   id           1367211 non-null  object 
 1   flag         1367211 non-null  int64  
 2   Dat_Flg1_Cd  1367211 non-null  object 
 3   Dat_Flg3_Cd  1367211 non-null  object 
 4   Trx_Cod1_Cd  1367211 non-null  int64  
 5   Trx_Cod2_Cd  1367211 non-null  int64  
 6   trx_tm       1367211 non-null  object 
 7   cny_trx_amt  1367211 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 83.4+ MB


In [6]:
trd_data.head()

,id,flag,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
0,UFDC88A,0,B,B,1,116,2019-06-20 07:15:28,-127.99
1,UFDC88A,0,B,B,1,116,2019-06-16 10:09:13,-55.88
2,UFDC88A,0,B,B,1,136,2019-05-14 16:11:32,-557.00
3,UFDC88A,0,B,B,1,136,2019-05-19 21:54:40,-77.80
4,UFDC88A,0,B,B,1,113,2019-06-18 08:23:59,-271.62


In [7]:
for col in trd_data.columns:
    print(trd_data[col].value_counts())

UBD9C48    5836
U8AE05B    2589
U0015B2    2490
U9E0C0D    2392
U7EF7EC    2237
           ... 
U0D701D       1
U6D7FB8       1
U236E05       1
UB5E3C5       1
U686247       1
Name: id, Length: 31993, dtype: int64
0    1152080
1     215131
Name: flag, dtype: int64
B    1068119
C     299092
Name: Dat_Flg1_Cd, dtype: int64
A    695630
B    661136
C     10445
Name: Dat_Flg3_Cd, dtype: int64
1    956662
3    239078
2    171471
Name: Trx_Cod1_Cd, dtype: int64
136    220378
132    180646
309     97218
308     77705
213     76653
111     70659
103     58876
102     55049
117     51679
134     45645
128     39153
310     37609
208     33827
209     26916
104     25921
131     24257
116     23696
123     20982
112     19309
135     19032
204     18265
101     14976
108     13932
113     10898
118     10726
133     10655
307      9858
130      9709
301      9127
201      6830
107      6560
302      4513
125      4345
207      3667
129      3338
114      3180
110      2991
109      2935
126      

In [63]:
beh_data = pd.read_csv('训练数据集/训练数据集_beh.csv')
beh_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934282 entries, 0 to 934281
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       934282 non-null  object
 1   flag     934282 non-null  int64 
 2   page_no  934282 non-null  object
 3   page_tm  934282 non-null  object
dtypes: int64(1), object(3)
memory usage: 28.5+ MB


In [9]:
for col in beh_data.columns:
    print(beh_data[col].value_counts())

U0015B2    3617
U8AE05B    3006
U8B3216    2255
U3C95E8    2125
UA4826C    1754
           ... 
UDFC46A       1
U4D8CD0       1
UB9975A       1
UB1CF80       1
U7CC42B       1
Name: id, Length: 11913, dtype: int64
0    793590
1    140692
Name: flag, dtype: int64
CQA    337400
CQE    151258
AAO    135763
TRN     67472
XAI     53678
MSG     45668
FTR     35838
SZA     17384
BWA     11581
CTR      9759
XAG      9453
CQD      8978
SZD      8335
CQC      7042
EGA      6279
GBA      5726
CQB      5117
LC0      3950
BWE      3911
JF2      1759
JJK      1401
EGB      1351
MTA      1046
FLS      1000
LCT       915
FDA       855
SYK       677
JJD       380
ZY1       306
Name: page_no, dtype: int64
2019/6/20 11:52    101
2019/6/6 9:25       99
2019/6/26 10:46     97
2019/6/27 16:05     95
2019/6/10 13:54     95
                  ... 
2019/6/23 1:29       1
2019/6/25 5:29       1
2019/6/29 3:47       1
2019/6/24 5:20       1
2019/6/3 4:07        1
Name: page_tm, Length: 39573, dtype: int64


# 提取特征

In [64]:
def is_na(data):
    return int(data == '\\N' or str(data) == '-1' or data == '~' or len(str(data)) == 0)
feature_tag = pd.DataFrame()
for col in user_data.columns:
    if '\\N' in user_data[col].value_counts().index or '-1' in user_data[col].value_counts().index or -1 in user_data[col].value_counts().index or '~' in user_data[col].value_counts().index:
        tmp = col + '_flag_0'
        feature_tag[tmp] = user_data[col].progress_apply(lambda x: is_na(x))


100%|████████████████████████████████████████████████████████████████████████| 39923/39923 [00:00<00:00, 677929.23it/s]


In [65]:
tmp_list = set(trd_data['id'].tolist())
tmp_list1 = set(beh_data['id'].tolist())
def is_inTrd(id):
    return int(id in tmp_list)
def is_inBeh(id):
    return int(id in tmp_list1)
feature_tag['is_inTrd'] = user_data['id'].progress_apply(lambda x: is_inTrd(x))
feature_tag['is_inBeh'] = user_data['id'].progress_apply(lambda x: is_inBeh(x))

100%|████████████████████████████████████████████████████████████████████████| 39923/39923 [00:00<00:00, 829930.16it/s]


In [39]:
def miss(x):
    return(sum(pd.isnull(x))/len(x))
feature_tag['miss_rate'] = user_data.progress_apply(lambda row:miss(row), axis=1)




  0%|                                                | 0/39923 [00:00<?, ?it/s]


  1%|▏                                   | 235/39923 [00:00<00:16, 2348.37it/s]


  2%|▋                                   | 809/39923 [00:00<00:13, 2854.03it/s]


  4%|█▎                                 | 1439/39923 [00:00<00:11, 3413.91it/s]


  5%|█▊                                 | 2055/39923 [00:00<00:09, 3940.42it/s]


  7%|██▎                                | 2674/39923 [00:00<00:08, 4421.94it/s]


  8%|██▉                                | 3286/39923 [00:00<00:07, 4822.64it/s]


 10%|███▍                               | 3944/39923 [00:00<00:06, 5241.79it/s]


 12%|████                               | 4597/39923 [00:00<00:06, 5570.55it/s]


 13%|████▌                              | 5234/39923 [00:00<00:05, 5787.38it/s]


 15%|█████                              | 5838/39923 [00:01<00:05, 5825.73it/s]


 16%|█████▋                             | 6457/39923 [00:01<00:05, 5929.27it/s]


 18%|██████▏ 

In [12]:
method = trd_data.loc[:, ['id', 'Dat_Flg3_Cd']]
x1 = method.groupby('id').agg(lambda x: x.value_counts().index[0]).reset_index()
feature_trd = pd.DataFrame(x1)


In [13]:
x2 = trd_data.loc[:, ['id', 'Dat_Flg1_Cd']]
B_num = x2[x2['Dat_Flg1_Cd'] == 'B'].groupby('id').count().reset_index()
C_num = x2[x2['Dat_Flg1_Cd'] == 'C'].groupby('id').count().reset_index()
new = pd.merge(feature_trd, B_num, how='left', on='id')
feature_trd['Dat_Flg1_Cd_B'] = new['Dat_Flg1_Cd']
new = pd.merge(feature_trd, C_num, how='left', on='id')
feature_trd['Dat_Flg1_Cd_C'] = new['Dat_Flg1_Cd']

In [14]:
feature_trd['Dat_Flg1_Cd_C'].fillna(1e-4, inplace = True)
feature_trd['Dat_Flg1_Cd_B'].fillna(1e-4, inplace = True)

In [15]:
feature_trd[feature_trd['Dat_Flg1_Cd_C'] == 0]['Dat_Flg1_Cd_C'] = 1e-4
feature_trd[feature_trd['Dat_Flg1_Cd_B'] == 0]['Dat_Flg1_Cd_B'] = 1e-4

In [16]:
money = trd_data.loc[:, ['id', 'cny_trx_amt']]
B_num = money[money['cny_trx_amt'] < 0].groupby('id').sum().reset_index()
B_num['cny_trx_amt'] = abs(B_num['cny_trx_amt'])
C_num = money[money['cny_trx_amt'] > 0].groupby('id').sum().reset_index()


In [17]:
new = pd.merge(feature_trd, B_num, how='left', on='id')
feature_trd['cny_trx_amt_B'] = new['cny_trx_amt']
new = pd.merge(feature_trd, C_num, how='left', on='id')
feature_trd['cny_trx_amt_C'] = new['cny_trx_amt']

In [18]:
feature_trd['cny_trx_amt_B'].fillna(0, inplace = True)
feature_trd['cny_trx_amt_C'].fillna(0, inplace = True)

In [19]:
new = trd_data.loc[:,['id', 'Trx_Cod2_Cd']].groupby(['id', 'Trx_Cod2_Cd']).size().reset_index()
x3 = new.groupby('id').count().loc[:, 'Trx_Cod2_Cd'].reset_index()
feature_trd['Trx_Cod2_Cd'] = x3['Trx_Cod2_Cd']

In [55]:
feature_trd

,id,Dat_Flg3_Cd,Trx_Cod2_Cd,trx/Flg1_B,trx/Flg1_C,Trx_Cod1_Cd
0,U000101,B,9,207.958684,559.195714,1
1,U00015C,B,6,7078.750000,9100.000000,2
2,U0001B8,A,14,260.028824,702.398000,3
3,U00025B,B,9,665.050541,14954.440000,1
4,U000437,B,13,2301.776857,5863.224545,3
...,...,...,...,...,...,...
31988,UFFF6BB,A,18,582.399390,475.909296,1
31989,UFFF7F4,B,22,575.064780,10101.184500,3
31990,UFFF9D8,B,1,998.000000,0.000000,2
31991,UFFFC56,A,13,6815.224848,8677.067308,3


In [20]:
feature_trd['trx/Flg1_B'] = feature_trd['cny_trx_amt_B']/feature_trd['Dat_Flg1_Cd_B']
feature_trd['trx/Flg1_C'] = feature_trd['cny_trx_amt_C']/feature_trd['Dat_Flg1_Cd_C']

In [21]:
feature_trd.drop(['Dat_Flg1_Cd_B', 'Dat_Flg1_Cd_C','cny_trx_amt_B','cny_trx_amt_C'], axis = 1, inplace = True)

In [16]:
B_max = money[money['cny_trx_amt'] < 0].groupby('id').max().reset_index()
B_max['cny_trx_amt'] = abs(B_num['cny_trx_amt'])
C_max = money[money['cny_trx_amt'] > 0].groupby('id').max().reset_index()

In [17]:
new = pd.merge(feature_trd, B_max, how='left', on='id')
feature_trd['cny_trx_amt_B_max'] = new['cny_trx_amt']
new = pd.merge(feature_trd, C_max, how='left', on='id')
feature_trd['cny_trx_amt_C_max'] = new['cny_trx_amt']

feature_trd['cny_trx_amt_B_max'].fillna(0, inplace = True)
feature_trd['cny_trx_amt_C_max'].fillna(0, inplace = True)

In [22]:
method1 = trd_data.loc[:, ['id', 'Trx_Cod1_Cd']]
x4 = method1.groupby(['id', 'Trx_Cod1_Cd']).size().reset_index()
feature_trd['Trx_Cod1_Cd'] = x4['Trx_Cod1_Cd']

In [23]:
trd_data['trx_tm'] = pd.to_datetime(trd_data['trx_tm'])
trd_data = trd_data.set_index('trx_tm')
this_mount = trd_data['2019-06']
this_mount = this_mount.loc[:,['id', 'cny_trx_amt']].groupby('id').sum().reset_index()
feature_trd['cny_trx_amt_thisMonth'] = this_mount['cny_trx_amt']
feature_trd['cny_trx_amt_thisMonth'].fillna(0, inplace = True)


In [24]:
this_mount = trd_data['2019-06']
this_mount = this_mount.loc[:,['id', 'Trx_Cod1_Cd']].groupby('id').count().reset_index()
feature_trd['trd_count'] = this_mount['Trx_Cod1_Cd']
feature_trd['trd_count'].fillna(0, inplace = True)

In [80]:
feature_trd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31993 entries, 0 to 31992
Data columns (total 8 columns):
id                       31993 non-null object
Dat_Flg3_Cd              31993 non-null object
Trx_Cod2_Cd              31993 non-null int64
trx/Flg1_B               31993 non-null float64
trx/Flg1_C               31993 non-null float64
Trx_Cod1_Cd              31993 non-null int64
cny_trx_amt_thisMonth    31993 non-null float64
trd_count                31993 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 2.0+ MB


In [25]:
beh_num = beh_data.groupby('id')['page_no'].count().reset_index()
beh_feature = pd.DataFrame(beh_num)


In [26]:
beh_cls = beh_data.loc[:, ['id', 'page_no']]
x5 = beh_cls.groupby('id').agg(lambda x: x.value_counts().index[0]).reset_index()
beh_feature['page_cls'] = x5['page_no']

# 测试集提取特征

In [27]:
user_data_test = pd.read_csv('评分数据集b/评分数据集_tag_b.csv')

In [28]:
feature_tag_test = pd.DataFrame()
for col in user_data.columns:
    if '\\N' in user_data[col].value_counts().index or '-1' in user_data[col].value_counts().index or -1 in user_data[col].value_counts().index or '~' in user_data[col].value_counts().index:
        tmp = col + '_flag_0'
        feature_tag_test[tmp] = user_data_test[col].progress_apply(lambda x: is_na(x))

#feature_tag_test['miss_rate'] = user_data_test.progress_apply(lambda row:miss(row), axis=1) 

100%|██████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 572933.65it/s]


In [34]:
tmp_list = set(trd_data_test['id'].tolist())
tmp_list1 = set(beh_data_test['id'].tolist())
feature_tag_test['is_inTrd'] = user_data_test['id'].progress_apply(lambda x: is_inTrd(x))
feature_tag_test['is_inBeh'] = user_data_test['id'].progress_apply(lambda x: is_inBeh(x))

100%|██████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 801816.86it/s]


In [35]:
#feature_tag.to_csv("data/feature_tag.csv",index=False)
feature_tag_test.to_csv("data/feature_tag_test_b.csv",index=False)

In [32]:
trd_data_test = pd.read_csv('评分数据集b/评分数据集_trd_b.csv')
trd_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142645 entries, 0 to 142644
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           142645 non-null  object 
 1   Dat_Flg1_Cd  142645 non-null  object 
 2   Dat_Flg3_Cd  142645 non-null  object 
 3   Trx_Cod1_Cd  142645 non-null  int64  
 4   Trx_Cod2_Cd  142645 non-null  int64  
 5   trx_tm       142645 non-null  object 
 6   cny_trx_amt  142645 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 7.6+ MB


In [21]:
for col in trd_data_test.columns:
    print(trd_data_test[col].value_counts())

U712783    4861
U9EF083    1836
U60405F    1257
U70EBFD    1093
UA94395    1014
           ... 
U7924B7       1
U93CAB1       1
UD0858D       1
U7DDF20       1
U9241B9       1
Name: id, Length: 4787, dtype: int64
B    166158
C     48657
Name: Dat_Flg1_Cd, dtype: int64
A    113014
B     99905
C      1896
Name: Dat_Flg3_Cd, dtype: int64
1    148166
3     38488
2     28161
Name: Trx_Cod1_Cd, dtype: int64
136    33279
132    31272
309    15325
308    12688
213    11629
111    11296
103     8998
117     7594
208     7482
102     7326
134     6665
128     5879
310     5459
104     3864
131     3805
209     3773
116     3301
123     3230
112     3100
135     3029
204     2821
101     2312
108     2181
307     1928
113     1743
301     1725
133     1642
118     1629
130     1429
201     1060
107     1017
302      797
125      696
207      531
110      526
114      469
129      465
109      425
210      381
126      364
306      360
205      292
127      233
122      178
304      118
105      1

In [36]:
method = trd_data_test.loc[:, ['id', 'Dat_Flg3_Cd']]
x1 = method.groupby('id').agg(lambda x: x.value_counts().index[0]).reset_index()
feature_trd_test = pd.DataFrame(x1)


In [37]:
x2 = trd_data_test.loc[:, ['id', 'Dat_Flg1_Cd']]
B_num = x2[x2['Dat_Flg1_Cd'] == 'B'].groupby('id').count().reset_index()
C_num = x2[x2['Dat_Flg1_Cd'] == 'C'].groupby('id').count().reset_index()
new = pd.merge(feature_trd_test, B_num, how='left', on='id')
feature_trd_test['Dat_Flg1_Cd_B'] = new['Dat_Flg1_Cd']
new = pd.merge(feature_trd_test, C_num, how='left', on='id')
feature_trd_test['Dat_Flg1_Cd_C'] = new['Dat_Flg1_Cd']

In [38]:
feature_trd_test['Dat_Flg1_Cd_C'].fillna(1e-4, inplace = True)
feature_trd_test['Dat_Flg1_Cd_B'].fillna(1e-4, inplace = True)

In [39]:
feature_trd_test[feature_trd_test['Dat_Flg1_Cd_C'] == 0]['Dat_Flg1_Cd_C'] = 1e-4
feature_trd_test[feature_trd_test['Dat_Flg1_Cd_B'] == 0]['Dat_Flg1_Cd_B'] = 1e-4

In [40]:
money = trd_data_test.loc[:, ['id', 'cny_trx_amt']]
B_num = money[money['cny_trx_amt'] < 0].groupby('id').sum().reset_index()
B_num['cny_trx_amt'] = abs(B_num['cny_trx_amt'])
C_num = money[money['cny_trx_amt'] > 0].groupby('id').sum().reset_index()

new = pd.merge(feature_trd_test, B_num, how='left', on='id')
feature_trd_test['cny_trx_amt_B'] = new['cny_trx_amt']
new = pd.merge(feature_trd_test, C_num, how='left', on='id')
feature_trd_test['cny_trx_amt_C'] = new['cny_trx_amt']

feature_trd_test['cny_trx_amt_B'].fillna(0, inplace = True)
feature_trd_test['cny_trx_amt_C'].fillna(0, inplace = True)

new = trd_data_test.loc[:,['id', 'Trx_Cod2_Cd']].groupby(['id', 'Trx_Cod2_Cd']).size().reset_index()
x3 = new.groupby('id').count().loc[:, 'Trx_Cod2_Cd'].reset_index()
feature_trd_test['Trx_Cod2_Cd'] = x3['Trx_Cod2_Cd']

In [41]:
feature_trd_test

,id,Dat_Flg3_Cd,Dat_Flg1_Cd_B,Dat_Flg1_Cd_C,cny_trx_amt_B,cny_trx_amt_C,Trx_Cod2_Cd
0,U000306,B,4.0,0.0001,217.00,0.00,2
1,U000B85,B,2.0,3.0000,617.00,631.19,3
2,U0013E0,A,266.0,27.0000,297601.78,324937.09,26
3,U00171D,B,4.0,0.0001,91.84,0.00,2
4,U002608,B,16.0,2.0000,933.30,3006.45,4
...,...,...,...,...,...,...,...
3185,U54A1A2,B,4.0,1.0000,1290.06,0.39,3
3186,U54C221,A,286.0,12.0000,35310.70,75389.22,21
3187,U54C3FC,A,8.0,9.0000,40990.48,59829.29,8
3188,U54C48A,B,2.0,1.0000,9900.00,51.53,2


In [42]:
feature_trd_test['trx/Flg1_B'] = feature_trd_test['cny_trx_amt_B']/feature_trd_test['Dat_Flg1_Cd_B']
feature_trd_test['trx/Flg1_C'] = feature_trd_test['cny_trx_amt_C']/feature_trd_test['Dat_Flg1_Cd_C']

In [43]:
feature_trd_test.drop(['Dat_Flg1_Cd_B', 'Dat_Flg1_Cd_C','cny_trx_amt_B','cny_trx_amt_C'], axis = 1, inplace = True)

In [44]:
trd_data_test['trx_tm'] = pd.to_datetime(trd_data_test['trx_tm'])
trd_data_test = trd_data_test.set_index('trx_tm')
this_mount = trd_data_test['2019-06']
this_mount = this_mount.loc[:,['id', 'cny_trx_amt']].groupby('id').sum().reset_index()
feature_trd_test['cny_trx_amt_thisMonth'] = this_mount['cny_trx_amt']
feature_trd_test['cny_trx_amt_thisMonth'].fillna(0, inplace = True)

In [45]:
this_mount = trd_data_test['2019-06']
this_mount = this_mount.loc[:,['id', 'Trx_Cod1_Cd']].groupby('id').count().reset_index()
feature_trd_test['trd_count'] = this_mount['Trx_Cod1_Cd']
feature_trd_test['trd_count'].fillna(0, inplace = True)

In [30]:
new = pd.merge(feature_trd_test, B_max, how='left', on='id')
feature_trd_test['cny_trx_amt_B_max'] = new['cny_trx_amt']
new = pd.merge(feature_trd_test, C_max, how='left', on='id')
feature_trd_test['cny_trx_amt_C_max'] = new['cny_trx_amt']

feature_trd_test['cny_trx_amt_B_max'].fillna(0, inplace = True)
feature_trd_test['cny_trx_amt_C_max'].fillna(0, inplace = True)

In [46]:
method1 = trd_data_test.loc[:, ['id', 'Trx_Cod1_Cd']]
x4 = method1.groupby(['id', 'Trx_Cod1_Cd']).size().reset_index()
feature_trd_test['Trx_Cod1_Cd'] = x4['Trx_Cod1_Cd']

In [47]:
feature_trd_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190 entries, 0 to 3189
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3190 non-null   object 
 1   Dat_Flg3_Cd            3190 non-null   object 
 2   Trx_Cod2_Cd            3190 non-null   int64  
 3   trx/Flg1_B             3190 non-null   float64
 4   trx/Flg1_C             3190 non-null   float64
 5   cny_trx_amt_thisMonth  3190 non-null   float64
 6   trd_count              3190 non-null   float64
 7   Trx_Cod1_Cd            3190 non-null   int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 199.5+ KB


In [48]:
feature_trd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31993 entries, 0 to 31992
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     31993 non-null  object 
 1   Dat_Flg3_Cd            31993 non-null  object 
 2   Trx_Cod2_Cd            31993 non-null  int64  
 3   trx/Flg1_B             31993 non-null  float64
 4   trx/Flg1_C             31993 non-null  float64
 5   Trx_Cod1_Cd            31993 non-null  int64  
 6   cny_trx_amt_thisMonth  31993 non-null  float64
 7   trd_count              31993 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 2.0+ MB


In [49]:
#feature_trd.to_csv("data/feature_trd.csv",index=False)
feature_trd_test.to_csv("data/feature_trd_test_b.csv",index=False)

In [33]:
beh_data_test = pd.read_csv('评分数据集b/评分数据集_beh_b.csv')
beh_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95669 entries, 0 to 95668
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          95669 non-null  object 
 1   page_no     95669 non-null  object 
 2   Unnamed: 2  95669 non-null  object 
 3   page_tm     0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 2.9+ MB


In [57]:
for col in beh_data_test.columns:
    print(beh_data_test[col].value_counts())

U60405F    1389
U5E0CFD    1190
U88936A    1047
U72166D     991
U600809     933
           ... 
UA71333       1
U9D5C9D       1
U5D9522       1
UBE0698       1
U7FF85A       1
Name: id, Length: 1844, dtype: int64
CQA    54221
CQE    24287
AAO    24015
TRN    10415
XAI     7637
MSG     6857
FTR     5331
SZA     2970
BWA     1743
CTR     1587
CQD     1470
SZD     1378
XAG     1363
CQC     1293
EGA      984
GBA      876
CQB      809
LC0      661
BWE      636
JJK      420
JF2      333
MTA      255
FLS      197
LCT      173
FDA      169
EGB      160
SYK      103
JJD       75
ZY1       60
Name: page_no, dtype: int64
2019-06-27 11:21:55    6
2019-06-05 17:54:06    5
2019-06-30 10:44:38    5
2019-06-11 17:12:24    5
2019-06-14 18:00:05    5
                      ..
2019-06-18 17:42:07    1
2019-06-03 14:26:11    1
2019-06-09 02:43:28    1
2019-06-11 16:06:33    1
2019-06-29 14:23:36    1
Name: Unnamed: 2, Length: 131588, dtype: int64
Series([], Name: page_tm, dtype: int64)


In [50]:
beh_num = beh_data_test.groupby('id')['page_no'].count().reset_index()
beh_feature_test = pd.DataFrame(beh_num)

beh_cls = beh_data_test.loc[:, ['id', 'page_no']]
x5 = beh_cls.groupby('id').agg(lambda x: x.value_counts().index[0]).reset_index()
beh_feature_test['page_cls'] = x5['page_no']

In [111]:
beh_feature_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1844 entries, 0 to 1843
Data columns (total 3 columns):
id          1844 non-null object
page_no     1844 non-null int64
page_cls    1844 non-null object
dtypes: int64(1), object(2)
memory usage: 43.3+ KB


In [51]:
#beh_feature.to_csv("data/feature_beh.csv",index=False)
beh_feature_test.to_csv("data/feature_beh_test_b.csv",index=False)